In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgbm
from sklearn.preprocessing import Normalizer

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
a = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/train_V2.csv')
b = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/test_V2.csv')

In [ ]:
a.shape

In [ ]:
b.shape

In [ ]:
a.drop(2744604, inplace=True)

In [ ]:
ticks = a['kills'].values
f,ax = plt.subplots(figsize = (10,10))
ax.set_xticklabels(ticks,rotation = 60)
sns.boxplot(x = 'kills',y = 'damageDealt',data = a)

In [ ]:
f,ax = plt.subplots(figsize = (10,10))
sns.scatterplot(x = 'kills',y = 'killPlace',data = a)

In [ ]:
no_matches = a.loc[:,'matchId'].nunique()
print(no_matches)

In [ ]:
match_types = a['matchType'].value_counts().reset_index()
match_types.columns = ['Type','Count']
print(match_types)

In [ ]:
ticks = match_types.Type.values
f,ax = plt.subplots(figsize = (10,10))
ax.set_xticklabels(ticks,rotation = 60)
sns.barplot(x = 'Type',y = 'Count',data = match_types)
plt.show()

In [ ]:
match_types2 = a['matchType'].value_counts().to_frame()

In [ ]:
agg_squads = match_types2.loc[['squad-fpp','squad','normal-squad-fpp','normal-squad'],'matchType'].sum()

In [ ]:
agg_duos = match_types2.loc[['duo-fpp','duo','normal-duo-fpp','normal-duo'],'matchType'].sum()

In [ ]:
agg_solos = match_types2.loc[['solo-fpp','solo','normal-solo-fpp','normal-solo'],'matchType'].sum()

In [ ]:
agg_df = pd.DataFrame([agg_squads,agg_duos,agg_solos],index = ['Squad','duo','solo'],columns = ['Count'])
print(agg_df)

In [ ]:
sns.distplot(a['numGroups'])
plt.show()

In [ ]:
f,ax = plt.subplots(figsize = (10,10))
sns.boxplot(x = 'DBNOs',y = 'kills',data = a)

In [ ]:
a[a['kills']>60][['Id','headshotKills','damageDealt','kills','killStreaks','longestKill','assists','matchType']]

In [ ]:
f,ax = plt.subplots(figsize = (10,10))
sns.countplot(a['headshotKills'])

In [ ]:
f,ax = plt.subplots(figsize = (10,10))
sns.countplot(a['DBNOs'])

In [ ]:
sns.distplot(a['longestKill'])

In [ ]:
print(a['longestKill'].mean())
print(a['longestKill'].quantile(0.95))
print(a['longestKill'].max())

In [ ]:
n = a['walkDistance']==0
print(n.sum())
k = a['rideDistance'] == 0
print(k.sum())
x = a['swimDistance'] == 0
print(x.sum())

In [ ]:
suspects = a.query('winPlacePerc == 1 & walkDistance == 0')
suspects.head()

In [ ]:
print(suspects['rideDistance'].max(),suspects['swimDistance'].max())

In [ ]:
print(a['weaponsAcquired'].mean())
print(a['weaponsAcquired'].min())
print(a['weaponsAcquired'].max())
print(a['weaponsAcquired'].quantile(0.99))

In [ ]:
plt.hist(a['weaponsAcquired'],range = (0,10),rwidth= 0.9)
plt.show()

In [ ]:
ax = sns.clustermap(a.corr(),annot = True,linewidths = 0.6,fmt = '.2f',figsize = (15,15))
plt.show()

In [ ]:
top10 = a[a['winPlacePerc']>0.9]
print(top10['kills'].mean())
print(top10['kills'].min())
print(top10['kills'].max())
print(top10['kills'].quantile(0.95))

In [ ]:
print(top10['longestKill'].mean())
print(top10['longestKill'].max())

In [ ]:
sns.clustermap(top10.corr(),annot = True,fmt = '.2f',figsize = (15,10))
plt.show()

In [ ]:
a.dropna(subset=["winPlacePerc"], inplace=True)

In [ ]:
x = a.drop(['Id','matchType','matchId','groupId','winPlacePerc'],axis = 1)

In [ ]:
x = x.iloc[:100000]

In [ ]:
x.shape

In [ ]:
y = a['winPlacePerc']

In [ ]:
y = y.iloc[:100000]

In [ ]:
y.shape

In [ ]:
model = lgbm.LGBMRegressor(max_depth = 30,learning_rate = 0.01,n_estimators = 250)
model.fit(x,y)

In [ ]:
lgbm.plot_importance(model)

In [ ]:
b = b.drop(['Id','matchType','matchId','groupId'],axis = 1)

In [ ]:
y_pred = model.predict(b)

In [ ]:
print(y_pred)

In [ ]:
plt.hist(y_pred,bins = 20,rwidth = 0.9)
plt.show()